In [69]:
# External imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os
import time

import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stl._stl import STL
import numpy as np
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import zscore, skew, kurtosis, probplot
from pandas.plotting import autocorrelation_plot, lag_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.signal import periodogram
from scipy.cluster.hierarchy import dendrogram, linkage
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor

import pandas as pd
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

import os
import time
from abc import ABC, abstractmethod
from enum import Enum
from typing import Union

import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from matplotlib import pyplot as plt
from sklearn import clone
from sklearn.ensemble import RandomForestRegressor as RFR, GradientBoostingRegressor, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV, train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.tree import DecisionTreeRegressor
from statsmodels.tsa.seasonal import seasonal_decompose
from tqdm.notebook import tqdm
from xgboost import XGBRegressor
from flaml import AutoML
import joblib
import h2o
from h2o.automl import H2OAutoML

In [70]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Long Notebook, Eirik Varnes (StudID: 543576) , Jathavaan Shankarr (StudID:544755) [77] SindreFinnesDetEksternData

Setting rules for libraries

In [71]:
# Warnings

# Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000)

# Reading files

In [72]:
# Features to remove
DATE_CALC: str = "date_calc"
DATE_FORECAST: str = "date_forecast"
ABSOLUTE_HUMIDITY: str = 'absolute_humidity_2m:gm3'
AIR_DENSITY: str = 'air_density_2m:kgm3'
CEILING_HEIGHT: str = 'ceiling_height_agl:m'
CLEAR_SKY_ENERGY: str = 'clear_sky_energy_1h:J'
CLEAR_SKY_RAD: str = 'clear_sky_rad:W'
CLOUD_BASE: str = 'cloud_base_agl:m'
DEW_OR_RIME: str = 'dew_or_rime:idx'
DEW_POINT: str = 'dew_point_2m:K'
DIFFUSE_RAD: str = 'diffuse_rad:W'
DIFFUSE_RAD_1H: str = 'diffuse_rad_1h:J'
DIRECT_RAD: str = 'direct_rad:W'
DIRECT_RAD_1H: str = 'direct_rad_1h:J'
EFFECTIVE_CLOUD_COVER: str = 'effective_cloud_cover:p'
ELEVATION: str = 'elevation:m'
FRESH_SNOW_12H: str = 'fresh_snow_12h:cm'
FRESH_SNOW_1H: str = 'fresh_snow_1h:cm'
FRESH_SNOW_24H: str = 'fresh_snow_24h:cm'
FRESH_SNOW_3H: str = 'fresh_snow_3h:cm'
FRESH_SNOW_6H: str = 'fresh_snow_6h:cm'
IS_DAY: str = 'is_day:idx'
IS_IN_SHADOW: str = 'is_in_shadow:idx'
MSL_PRESSURE: str = 'msl_pressure:hPa'
PRECIP_5MIN: str = 'precip_5min:mm'
PRECIP_TYPE_5MIN: str = 'precip_type_5min:idx'
PRESSURE_100M: str = 'pressure_100m:hPa'
PRESSURE_50M: str = 'pressure_50m:hPa'
PROB_RIME: str = 'prob_rime:p'
RAIN_WATER: str = 'rain_water:kgm2'
RELATIVE_HUMIDITY: str = 'relative_humidity_1000hPa:p'
SFC_PRESSURE: str = 'sfc_pressure:hPa'
SNOW_DENSITY: str = 'snow_density:kgm3'
SNOW_DEPTH: str = 'snow_depth:cm'
SNOW_DRIFT: str = 'snow_drift:idx'
SNOW_MELT_10MIN: str = 'snow_melt_10min:mm'
SNOW_WATER: str = 'snow_water:kgm2'
SUN_AZIMUTH: str = 'sun_azimuth:d'
SUN_ELEVATION: str = 'sun_elevation:d'
SUPER_COOLED_LIQUID_WATER: str = 'super_cooled_liquid_water:kgm2'
T_1000HPA: str = 't_1000hPa:K'
TOTAL_CLOUD_COVER: str = 'total_cloud_cover:p'
VISIBILITY: str = 'visibility:m'
WIND_SPEED_10M: str = 'wind_speed_10m:ms'
WIND_SPEED_U_10M: str = 'wind_speed_u_10m:ms'
WIND_SPEED_V_10M: str = 'wind_speed_v_10m:ms'
WIND_SPEED_W_1000HPA: str = 'wind_speed_w_1000hPa:ms'

TIME = 'time'
PV_MEASUREMENT = 'pv_measurement'

In [73]:
# Functions
def read_parquet(filepath: str) -> pd.DataFrame:
    dataframe: pd.DataFrame = pd.read_parquet(filepath)
    if DATE_CALC in dataframe.columns:
        dataframe = dataframe.drop(columns=[DATE_CALC])

    return dataframe


def concat_observed_and_estimated_train_data(
        X_observed: pd.DataFrame,
        X_estimated: pd.DataFrame
) -> pd.DataFrame:
    X_observed: pd.DataFrame = X_observed.copy()
    X_estimated: pd.DataFrame = X_estimated.copy()

    assert X_observed.shape[1] == X_estimated.shape[1]

    concatenated_dataframe: pd.DataFrame = pd.concat(
        objs=[X_observed, X_estimated],
        axis=0
    ).sort_values(
        by=DATE_FORECAST,
        ascending=True
    ).reset_index(drop=True)

    return concatenated_dataframe

In [74]:
# Data transfer object for location data
class LocationData:
    def __init__(self, train_data, target_data, test_data) -> None:
        self.train_data = train_data
        self.target_data = target_data
        self.test_data = test_data
        
        self.original_train_data = train_data
        self.original_target_data = target_data
        self.original_test_data = test_data

    def __repr__(self) -> str:
        return f"Train: {self.train_data.shape} [No. NaN: {self.train_data.isna().sum().sum()}], Target: {self.target_data.shape} [No. NaN: {self.target_data.isna().sum().sum()}], Test: {self.test_data.shape} [No. NaN: {self.test_data.isna().sum().sum()}]"

In [75]:
observed_train = read_parquet("data/A/X_train_observed.parquet")
estimated_train = read_parquet("data/A/X_train_estimated.parquet")
target_data = read_parquet("data/A/train_targets.parquet")
test_data = read_parquet("data/A/X_test_estimated.parquet")

train_data = concat_observed_and_estimated_train_data(
    X_observed=observed_train,
    X_estimated=estimated_train
)

In [76]:
A = LocationData(
    train_data=train_data,
    target_data=target_data,
    test_data=test_data
)

In [77]:
observed_train = read_parquet("data/B/X_train_observed.parquet")
estimated_train = read_parquet("data/B/X_train_estimated.parquet")
target_data = read_parquet("data/B/train_targets.parquet")
test_data = read_parquet("data/B/X_test_estimated.parquet")

train_data = concat_observed_and_estimated_train_data(
    X_observed=observed_train,
    X_estimated=estimated_train
)

B = LocationData(
    train_data=train_data,
    target_data=target_data,
    test_data=test_data
)

In [78]:
observed_train = read_parquet("data/C/X_train_observed.parquet")
estimated_train = read_parquet("data/C/X_train_estimated.parquet")
target_data = read_parquet("data/C/train_targets.parquet")
test_data = read_parquet("data/C/X_test_estimated.parquet")

train_data = concat_observed_and_estimated_train_data(
    X_observed=observed_train,
    X_estimated=estimated_train
)

In [79]:
C = LocationData(
    train_data=train_data,
    target_data=target_data,
    test_data=test_data
)

In [80]:
display(A)
display(B)
display(C)

Train: (136245, 46) [No. NaN: 168040], Target: (34085, 2) [No. NaN: 0], Test: (2880, 46) [No. NaN: 3971]

Train: (134505, 46) [No. NaN: 158811], Target: (32848, 2) [No. NaN: 4], Test: (2880, 46) [No. NaN: 3912]

Train: (134401, 46) [No. NaN: 157326], Target: (32155, 2) [No. NaN: 6060], Test: (2880, 46) [No. NaN: 4104]

# Preprocessing
Function to display info about dataframe, used for debugging
## Defining functions for preprocessing

In [81]:
def display_info(df: pd.DataFrame, show_nan_dataframe: bool = False) -> None:
    if not isinstance(df, pd.DataFrame):
        raise TypeError(f"Expected dataframe, got {type(df)}")

    obj_type = type(df)
    shape = df.shape
    nan_sum = df.isna().sum().sum()
    nan_indexes = np.where(df.isna().any(axis=1))[0]
    nan_df = df.iloc[nan_indexes]
    print(f"Type: {obj_type}, Shape: {shape}, NaN sum: {nan_sum}")
    if show_nan_dataframe:
        display(nan_df)
    else:
        display(df)

Function to remove features and replace `NaN` values with zero

In [82]:
def remove_feature(dataframe: pd.DataFrame, *features: str) -> pd.DataFrame:
    dataframe = dataframe.drop(columns=[*features])
    return dataframe


def replace_nan_with_zero(dataframe: pd.DataFrame, feature: str) -> pd.DataFrame:
    dataframe = dataframe.copy()
    dataframe[[feature]] = dataframe[[feature]].fillna(0)
    return dataframe

Function to resample data by categorizing them
- Features to average
- Features to pick a value
- Features to change redefine `NaN`.

In [ ]:
def sample_by_selection(dataframe: pd.DataFrame, *features: str) -> pd.DataFrame:
    reduced_dataframe = dataframe[[*features]]
    if reduced_dataframe.isna().sum().sum() > 0:
        print("[ERROR] Sample by selection got dataframe with NaN values")

    resampler = reduced_dataframe.resample('H')
    reduced_dataframe = resampler.last()

    """
    # Dropping NaN values that were introduced by resampling
    reduced_dataframe = reduced_dataframe.dropna(axis=0, how='any')
    """

    return reduced_dataframe


def sample_by_mean(dataframe: pd.DataFrame, *features: str) -> pd.DataFrame:
    reduced_dataframe = dataframe[[*features]]
    if reduced_dataframe.isna().sum().sum() > 0:
        print("[ERROR] Sample by mean got dataframe with NaN values")

    resampler = reduced_dataframe.resample('H')
    reduced_dataframe = resampler.mean()

    """
    # Dropping NaN values that were introduced by resampling
    reduced_dataframe = reduced_dataframe.dropna(axis=0, how='any')
    """

    return reduced_dataframe


def sample_by_sum(dataframe: pd.DataFrame, *features: str) -> pd.DataFrame:
    reduced_dataframe = dataframe[[*features]]
    if reduced_dataframe.isna().sum().sum() > 0:
        print("[ERROR] Sample by mean got dataframe with NaN values")

    resampler = reduced_dataframe.resample('H')
    reduced_dataframe = resampler.sum()

    """
    # Dropping NaN values that were introduced by resampling
    reduced_dataframe = reduced_dataframe.dropna(axis=0, how='any')
    """

    return reduced_dataframe


def sample_cloud_base(dataframe: pd.DataFrame) -> pd.DataFrame:
    cloud_base_frame = dataframe[[CLOUD_BASE]]

    filled_frame = cloud_base_frame.copy()
    filled_frame = filled_frame.fillna(-100_000)

    resampler = filled_frame.resample('H')
    cloud_base_frame = resampler.mean()

    cloud_bases = cloud_base_frame[CLOUD_BASE].to_numpy()
    negative_indexes = np.where(cloud_bases < 0)[0]
    cloud_bases[negative_indexes] = -666
    cloud_base_frame[CLOUD_BASE] = cloud_bases

    # Dropping NaN values that were introduced by resampling

    reduced_dataframe = cloud_base_frame.copy()
    """
    reduced_dataframe = reduced_dataframe.dropna(axis=0, how='any')

    if reduced_dataframe.isna().sum().sum() > 0:
        print("[ERROR] Cloud base still has NaN values after resampling")
    """

    return reduced_dataframe


def merge_frames(*dataframes: pd.DataFrame) -> pd.DataFrame:
    indexes = [df.index for df in dataframes]

    intersecting_indexes = list(set(indexes[0]).intersection(*indexes))
    filtered_dataframes = [df.loc[intersecting_indexes] for df in dataframes]

    return pd.concat(filtered_dataframes, axis=1)

Functions to synchronze dates in weather data and target data

In [ ]:
def synchronize_timestamps(train_data: pd.DataFrame, target_data: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    if not isinstance(train_data.index, pd.DatetimeIndex):
        raise IndexError("Train data does not have DateTimeIndex")

    if not isinstance(target_data.index, pd.DatetimeIndex):
        raise IndexError("Target data does not have DateTimeIndex")

    train_data.sort_index(inplace=True)
    target_data.sort_index(inplace=True)

    train_dates = train_data.index.values
    target_dates = target_data.index.values

    # Finding intersecting dates and filtering dataframes
    intersecting_dates = np.intersect1d(train_dates, target_dates)
    train_data = train_data.loc[intersecting_dates]
    target_data = target_data.loc[intersecting_dates]

    return train_data, target_data


The dateset contains many constant values for some features, any row with constant values is removed. If there is are at least 24 rows with constant values, the rows are removed.
- -666 in ceiling height should not be removed as this is added in earlier 

In [ ]:
def remove_constant_measurements(
        train_data: pd.DataFrame,
        target_data: pd.DataFrame,
        THRESHOLD: int = 24
) -> tuple[pd.DataFrame, pd.DataFrame]:
    indexes = []
    count = 0
    previous_value = None
    start_index = None

    # Setting date indexes as columns and resetting index
    train_data[DATE_FORECAST] = train_data.index.values
    target_data[TIME] = target_data.index.values

    train_data.reset_index(drop=True, inplace=True)
    target_data.reset_index(drop=True, inplace=True)

    # Assuming y is a DataFrame and has a column for pv_measurement values
    pv_measurements = target_data[PV_MEASUREMENT].to_numpy()

    # Identifying indexes of consecutive constant values
    for index, value in enumerate(pv_measurements):
        if value == previous_value:
            count += 1
            if count == 1:
                start_index = index - 1
            if count >= THRESHOLD and value > -0.1:
                indexes.extend(range(start_index, index + 1))
        else:
            count = 0
        previous_value = value

    # Assuming y has a column for dates and X has a similar column to filter on
    dates = target_data.loc[indexes, TIME].unique()

    filtered_train_data = train_data[~train_data[DATE_FORECAST].isin(dates)]
    filtered_target_data = target_data[~target_data[TIME].isin(dates)]

    # Switching back to date as index
    filtered_train_data = filtered_train_data.set_index(DATE_FORECAST)
    filtered_target_data = filtered_target_data.set_index(TIME)

    return filtered_train_data, filtered_target_data

## Adding lag features, this has to come here because of the resampling

In [ ]:
def add_lag_features(dataframe: pd.DataFrame, features: list[str], lags: list[int]) -> tuple[pd.DataFrame, list[str]]:
    dataframe = dataframe.copy()
    lag_feature_names: list[str] = []
    for feature in features:
        for lag in lags:
            lag_feature_name = f"{feature}_lag_{lag}"
            dataframe[lag_feature_name] = dataframe[feature].shift(lag)
            lag_feature_names.append(lag_feature_name)

    return dataframe, lag_feature_names


def add_lead_features(dataframe: pd.DataFrame, features: list[str], lags: list[int]) -> tuple[pd.DataFrame, list[str]]:
    dataframe = dataframe.copy()
    lead_feature_names: list[str] = []
    for feature in features:
        for lag in lags:
            lead_feature_name = f"{feature}_lead_{lag}"
            dataframe[lead_feature_name] = dataframe[feature].shift(-lag)
            lead_feature_names.append(lead_feature_name)

    return dataframe, lead_feature_names

Pipeline for preprocessing that is common for both train and test data

In [ ]:
def common_pre_processing(dataframe: pd.DataFrame) -> pd.DataFrame:
    dataframe = remove_feature(dataframe, SNOW_DRIFT, CEILING_HEIGHT, SNOW_DENSITY)
    dataframe = dataframe.set_index(DATE_FORECAST)

    feature_to_lag_and_lead: str = DIRECT_RAD
    # Adding lag and lead features
    dataframe, lag_features = add_lag_features(dataframe, [feature_to_lag_and_lead], [1, 2, 3])
    dataframe, lead_features = add_lead_features(dataframe, [feature_to_lag_and_lead], [1, 2, 3])

    # Resample data to hours
    selection_features = [
        CLEAR_SKY_ENERGY,  # Maybe move
        DIRECT_RAD_1H,  # MAYBE MOVE
        DIFFUSE_RAD_1H,  # Maybe move
        FRESH_SNOW_1H,
        FRESH_SNOW_3H,
        FRESH_SNOW_6H,
        FRESH_SNOW_12H,
        FRESH_SNOW_24H,
        DEW_OR_RIME,
        IS_DAY,
        IS_IN_SHADOW,
        PRECIP_TYPE_5MIN,
        ELEVATION,
        *lag_features,
        *lead_features,
    ]

    mean_features = [
        ABSOLUTE_HUMIDITY,
        CLEAR_SKY_RAD,
        DIFFUSE_RAD,
        DIRECT_RAD,
        AIR_DENSITY,
        DEW_POINT,
        MSL_PRESSURE,
        PRESSURE_100M,
        PRESSURE_50M,
        SFC_PRESSURE,
        RELATIVE_HUMIDITY,
        T_1000HPA,
        SUN_AZIMUTH,
        SUN_ELEVATION,
        EFFECTIVE_CLOUD_COVER,
        TOTAL_CLOUD_COVER,
        VISIBILITY,
        WIND_SPEED_10M,
        WIND_SPEED_U_10M,
        WIND_SPEED_V_10M,
        WIND_SPEED_W_1000HPA,
        SNOW_MELT_10MIN,
        PROB_RIME,
        SUPER_COOLED_LIQUID_WATER,
        SNOW_DEPTH,
    ]

    sum_features = [
        SNOW_WATER,
        PRECIP_5MIN,
        RAIN_WATER,
    ]

    selection_df = sample_by_selection(dataframe, *selection_features)
    mean_df = sample_by_mean(dataframe, *mean_features)
    sum_df = sample_by_sum(dataframe, *sum_features)
    ceiling_height_df = sample_cloud_base(dataframe)
    dataframe = merge_frames(selection_df, mean_df, ceiling_height_df, sum_df)

    dataframe = dataframe.dropna(axis=0, how='any')

    return dataframe

### Preprocessing pipelines for train and test data

In [ ]:
def pre_process_train_data(train_data: pd.DataFrame, target_data: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    train_data = common_pre_processing(dataframe=train_data)

    # Setting time as index for target data
    target_data[TIME] = pd.to_datetime(target_data[TIME])
    target_data = target_data.set_index(TIME)

    # Preprocessing that affects both train and target data
    train_data, target_data = synchronize_timestamps(train_data=train_data, target_data=target_data)
    train_data, target_data = remove_constant_measurements(train_data=train_data, target_data=target_data)

    return train_data, target_data


def pre_process_test_data(test_data: pd.DataFrame, columns: np.ndarray) -> pd.DataFrame:
    test_data = common_pre_processing(dataframe=test_data)
    if np.any(test_data.columns != columns):
        raise ValueError("Test data does not have the same columns as train data")
    return test_data

## Running preprocessing
Updating data objects for each location

In [ ]:
train_data, target_data = pre_process_train_data(train_data=A.train_data, target_data=A.target_data)
test_data = pre_process_test_data(test_data=A.test_data, columns=train_data.columns)
A.train_data = train_data
A.target_data = target_data
A.test_data = test_data

In [ ]:
train_data, target_data = pre_process_train_data(train_data=B.train_data, target_data=B.target_data)
test_data = pre_process_test_data(test_data=B.test_data, columns=train_data.columns)
B.train_data = train_data
B.target_data = target_data
B.test_data = test_data

In [ ]:
train_data, target_data = pre_process_train_data(train_data=C.train_data, target_data=C.target_data)
test_data = pre_process_test_data(test_data=C.test_data, columns=train_data.columns)
C.train_data = train_data
C.target_data = target_data
C.test_data = test_data

In [ ]:
display(A)
display(B)
display(C)


# Extended preprocessing
As shown in the location summaries, we can see that there are many `NaN` values in the target data for location `B` and  some for location `C`. This needs to be handles as we loose valuable data if we remove all rows with `NaN` values. This is done by using data from the other location to fill in the missing values. We can safley assume that location `B` and `C` is the same location based on correlation between the features and measurements. They also tend to follow the same trends in the data.
Defining defining functions to impute `NaN` values using data from other locations.

In [ ]:
def impute_target_data(
        to_df: pd.DataFrame,
        from_df: pd.DataFrame,
        SCALE_FACTOR: int = 1,
        INTERCEPT: float = 0
) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Assigning variables to index column and pv_measurement column in each dataframe
    to_dates = to_df.index.values
    to_pv_measurements = to_df[PV_MEASUREMENT].to_numpy()
    from_dates = from_df.index.values
    from_pv_measurements = from_df[PV_MEASUREMENT].to_numpy()

    # Finding indexes of NaN values that we want to impute
    nan_to_impute_indexes = np.where(np.isnan(to_pv_measurements))[0]
    nan_to_impute_dates = to_dates[nan_to_impute_indexes]

    # Finding the indexes of the dates we want to impute in the from dataframe (source)
    source_df_date_indexes = []
    for nan_date in nan_to_impute_dates:
        source_nan_index = np.where(from_dates == nan_date)[0]
        if len(source_nan_index) > 0:
            source_df_date_indexes.append(source_nan_index[0])

    source_df_date_indexes = np.array(source_df_date_indexes, dtype=np.int64)
    source_dates = from_dates[source_df_date_indexes]
    source_pv_measurements = from_pv_measurements[source_df_date_indexes]
    source_pv_measurements = source_pv_measurements * SCALE_FACTOR + INTERCEPT

    # Finding indexes of NaN values in from data
    to_df.loc[source_dates, PV_MEASUREMENT] = source_pv_measurements

    return to_df, from_df


def impute_missing_dates(
        from_location: LocationData,
        to_location: LocationData,
        SCALE_FACTOR: int = 1,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    from_train = from_location.train_data
    from_target = from_location.target_data
    to_train = to_location.train_data
    to_target = to_location.target_data

    # Missing dates 
    missing_dates = np.setdiff1d(from_target.index.values, to_target.index.values)

    # Defining date ranges for both dataframes
    from_start_date = from_target.index.values[0]
    from_end_date = from_target.index.values[-1]

    to_start_date = to_target.index.values[0]
    to_end_date = to_target.index.values[-1]

    from_dates = np.arange(from_start_date, from_end_date + np.timedelta64(1, 'h'), dtype='datetime64[h]')
    to_dates = np.arange(to_start_date, to_end_date + np.timedelta64(1, 'h'), dtype='datetime64[h]')

    to_target_frame_with_full_date_range = pd.DataFrame(index=pd.DatetimeIndex(to_dates), columns=to_target.columns)
    to_target_frame_with_full_date_range.loc[to_target.index.values] = to_target.values
    to_target_frame_with_full_date_range.loc[missing_dates] = from_target.loc[missing_dates].values * SCALE_FACTOR

    to_train_frame_with_fuill_date_range = pd.DataFrame(index=pd.DatetimeIndex(to_dates), columns=to_train.columns)
    to_train_frame_with_fuill_date_range.loc[to_train.index.values] = to_train.values
    to_train_frame_with_fuill_date_range.loc[missing_dates] = from_train.loc[missing_dates].values

    return to_train_frame_with_fuill_date_range, to_target_frame_with_full_date_range, from_train, from_target

Pipeline for imputation of target data for B and C

In [ ]:
def target_data_imputation(B: LocationData, C: LocationData) -> None:
    target_B = B.target_data.copy()
    target_C = C.target_data.copy()

    target_B, target_C = impute_target_data(to_df=target_B, from_df=target_C,
                                            SCALE_FACTOR=1.10387868)  # 1.10387868 , 1.2
    target_C, target_B = impute_target_data(to_df=target_C, from_df=target_B, SCALE_FACTOR=0.82386204)  # 0.82386204

    """
    train_B, target_B, train_C, target_C = impute_missing_dates(to_location=B, from_location=C, SCALE_FACTOR=1.2)
    train_C, target_C, train_B, target_B = impute_missing_dates(to_location=C, from_location=B, SCALE_FACTOR=0.5)
    """

    # Dropping rows with NaN values, this is done as we have imputed all NaN values and any remaining NaN values are due to missing data in both locations
    target_B.dropna(axis=0, how='any', inplace=True)
    target_C.dropna(axis=0, how='any', inplace=True)

    train_B, target_B = synchronize_timestamps(train_data=B.train_data, target_data=target_B)
    train_C, target_C = synchronize_timestamps(train_data=C.train_data, target_data=target_C)

    B.train_data = train_B
    B.target_data = target_B
    C.target_data = target_C
    C.train_data = train_C


def drop_nan_values_from_target_data(location: LocationData) -> None:
    target_data = location.target_data.copy()
    target_data.dropna(axis=0, how='any', inplace=True)

    train_data = location.train_data.copy()
    train_data, target_data = synchronize_timestamps(train_data, target_data)

    location.train_data = train_data
    location.target_data = target_data

## Target value imputation by borrowing data beteween B and C

In [ ]:
#target_data_imputation(B, C)

In [ ]:

drop_nan_values_from_target_data(A)
drop_nan_values_from_target_data(B)
drop_nan_values_from_target_data(C)


In [ ]:
display(A)
display(B)
display(C)

## Synchronizing timestamps between train and target data

In [ ]:
display(A)
display(B)
display(C)

# Feature engineering
## Seasonality captured by sine and cosine
Function to create sinusoidal features

In [ ]:
def create_sinusoidal_features(df: pd.DataFrame, hour_col: str = 'hour', day_col: str = 'day_of_year') -> pd.DataFrame:
    df = df.copy()
    dates = df.index.values
    df['date_forecast'] = dates
    df.reset_index(drop=True, inplace=True)

    # Create hour and day_of_year columns
    df['hour'] = df['date_forecast'].dt.hour
    df['day_of_year'] = df['date_forecast'].dt.dayofyear

    # Extract week number using isocalendar()
    df['week_of_year'] = df['date_forecast'].dt.isocalendar().week
    df['month'] = df['date_forecast'].dt.month

    # Constants for sinusoidal functions
    hours_in_day = 24
    days_in_year = 365.25  # Accounting for leap years
    weeks_in_year = 52
    months_in_year = 12

    # Create sinusoidal features based on the hour of the day
    df['sin_hour'] = np.sin(2 * np.pi * (df[hour_col] - 18) / hours_in_day)
    df['cos_hour'] = np.cos(2 * np.pi * (df[hour_col] - 18) / hours_in_day)

    # Create sinusoidal features based on the day of the year
    df['sin_day'] = np.sin(2 * np.pi * (df[day_col] - 355) / days_in_year)
    df['cos_day'] = np.cos(2 * np.pi * (df[day_col] - 355) / days_in_year)

    # Create sinusoidal features based on the week of the year
    df['sin_week'] = np.sin(2 * np.pi * df['week_of_year'] / weeks_in_year)
    df['cos_week'] = np.cos(2 * np.pi * df['week_of_year'] / weeks_in_year)

    # Create sinusoidal features based on the month
    df['sin_month'] = np.sin(2 * np.pi * df['month'] / months_in_year)
    df['cos_month'] = np.cos(2 * np.pi * df['month'] / months_in_year)

    dates = df['date_forecast'].to_numpy()
    df.index = dates
    df.drop(columns=['hour', 'day_of_year', 'week_of_year', 'month', 'date_forecast'], inplace=True)

    return df


def create_sinusoidal_features_for_location(location: LocationData) -> None:
    train_data = location.train_data.copy()
    test_data = location.test_data.copy()

    train_data = create_sinusoidal_features(train_data)
    test_data = create_sinusoidal_features(test_data)

    train_data.sort_index(inplace=True)
    test_data.sort_index(inplace=True)

    location.train_data = train_data
    location.test_data = test_data

### Adding seasonality


In [ ]:
create_sinusoidal_features_for_location(A)
create_sinusoidal_features_for_location(B)
create_sinusoidal_features_for_location(C)

In [ ]:
display(A)
display(B)
display(C)

### Scaling start of ceiling height 

In [ ]:
import numpy as np
from scipy.stats import percentileofscore


def match_quantiles(series_before, series_after):
    quantiles_after = np.percentile(series_after, np.arange(0, 101, 1))

    def find_quantile_match(value, quantiles):
        percentile = percentileofscore(series_before, value, kind='rank')
        return np.percentile(quantiles, percentile)

    return series_before.apply(lambda x: find_quantile_match(x, quantiles_after))


def scale_ceiling_height(location, index):
    train_data = location.train_data.copy()
    meter_data = train_data.iloc[index:]

    good = train_data.iloc[index:][CEILING_HEIGHT]
    bad = train_data.iloc[:index][CEILING_HEIGHT]

    matched_series = match_quantiles(good, bad)

    train_data.loc[:index, CEILING_HEIGHT] = matched_series
    scaled_data = train_data.iloc[:index]

    merged_data = pd.concat([scaled_data, meter_data], axis=0)
    merged_data.sort_index(inplace=True)

    location.train_data = merged_data


def scale_ceiling_height_by_multiplication(location: LocationData, index: int) -> None:
    feet_data = location.train_data.iloc[:index][CEILING_HEIGHT]
    meter_data = location.train_data.iloc[index:][CEILING_HEIGHT]

    neg_feet_indexes = np.where(feet_data < 0)[0]
    feet_data_in_meter = feet_data * 0
    feet_data_in_meter[neg_feet_indexes] = -666

    merged_ceiling_height = pd.concat([feet_data_in_meter, meter_data], axis=0)
    merged_ceiling_height.sort_index(inplace=True)

    location.train_data[CEILING_HEIGHT] = merged_ceiling_height

In [ ]:
def plot_by_feature(location: LocationData, feature: str) -> None:
    train_data = location.train_data.copy()

    train_data.reset_index(drop=True, inplace=True)
    train_data = train_data[[feature]]

    plt.figure(figsize=(30, 10))
    plt.plot(train_data, label='Train')
    plt.legend()
    plt.show()

## Outlier detection and removal (Not used) 

In [ ]:
def remove_outliers_IQR(
            train_data,
            target_data,
            column,
            daily_seasonality=24,
            monthly_seasonality=730,
            upper_threshold=3,
            lower_threshold=3
    ) -> tuple[pd.DataFrame, pd.DataFrame]:
        result_daily = seasonal_decompose(train_data[column], model='additive', period=daily_seasonality)
        daily_adjusted = result_daily.resid.dropna()

        result_monthly = seasonal_decompose(daily_adjusted, model='additive', period=monthly_seasonality)
        monthly_adjusted = result_monthly.resid.dropna()

        rolling_mean = monthly_adjusted.rolling(window=daily_seasonality, center=True).mean()
        rolling_std = monthly_adjusted.rolling(window=daily_seasonality, center=True).std()

        upper_bound = rolling_mean + upper_threshold * rolling_std
        lower_bound = rolling_mean - lower_threshold * rolling_std

        outliers = ((monthly_adjusted > upper_bound) | (monthly_adjusted < lower_bound)).reindex(
            train_data.index,
            fill_value=False
        )

        cleaned_train_data = train_data[~outliers]
        cleaned_target_data = target_data[~outliers]

        return cleaned_train_data, cleaned_target_data

    
def remove_outliers_LOF(train_data, target_data,
                            n_neighbors=2,
                            contamination='auto',
                            drop_columns=['time']):
        features_for_lof = target_data.drop(drop_columns, axis=1)

        lof = LocalOutlierFactor(n_neighbors=n_neighbors, novelty=False, contamination=contamination)

        lof_labels = lof.fit_predict(features_for_lof)

        outliers_mask = lof_labels == -1
        cleaned_target_data = target_data[~outliers_mask]
        cleaned_train_data = train_data[~outliers_mask]

        return cleaned_train_data, cleaned_target_data

# 2 EDA 

# Weather Data and Solar Energy Production Analysis

The dataset consists of different weather phenomenas and solar energy production data. Weather data, in various units, include both instantaneous and cumulative values for every 15 min. The train data consists of observed and estimated weather conditions across two distinct, non-overlapping periods. We have data on energy production from a solar panel installation, collected from different locations during approximately the same period.

# Domain Knowledge: Weather Data and Energy Production

1. **Temperature**: 
   - Temperature directly impacts photo voltaic cell efficiency, and therefore energy production.
   - Solar panels usually perform better at moderate temperatures.

2. **Sunlight and Solar Radiation**: 
   - Important for solar energy production.
   - Includes direct sunlight and diffused sky radiation.
   - Affected by cloud cover blocking the solar panels.

3. **Wind**: 
   - Cooling effect can increase efficiency, but strong winds may damage installations.
   - High wind speeds can cause dust and debris to accumulate on panels, reducing efficiency.
   - High wind speed cause more change in other weather factors. 

4. **Precipitation**: 
   - Rainfall can clean solar panels, enhancing efficiency.
   - Heavy snowfall can obstruct panels, reducing production.

5. **Humidity**: 
   - High humidity can lead to material degradation and connection issues.
   - High humidity is often related to cloud cover, which reduces solar radiation.

6. **Atmospheric Pressure**: 
   - Indicates weather pattern changes, indirectly affecting solar production.

7. **Cloud Cover**: 
   - Impacts the amount of solar radiation received.
   - Can diffuse or block sunlight.
   - May increase radiative forcing, which can increase temperatures.

8. **Air Quality**: 
   - Particulates and pollutants reduce solar radiation efficiency by blocking sunlight from cells.

9. **Seasonal Variations**: 
   - Sun angle changes affect daylight hours and solar radiation intensity throughout the year.

10. **Geographical Differences**: 
    - Location affects sunlight amount due to latitude, altitude, and local climates.


## Considerations and Remarks for Data Analysis
- **Data Synchronization:** Align weather data timestamps with energy production timestamps.
- **Unit Consistency:** Keep units in mind for meaningful comparisons.
- **Seasonal Analysis:** Consider seasonal variations in both weather patterns and solar energy production.


## 2.1 General Statistics

In [ ]:
def describe_numerical_data(df):
    
    description = df.describe(percentiles=[.25, .5, .75], include='all').transpose()
    
    # Adding additional statistics
    description['#NaN'] = df.isna().sum()
    description['#Unique'] = df.nunique()
    
    # Renaming the percentiles for clarity
    description = description.rename(columns={'25%': 'Q1 (25th Percentile)', '50%': 'Median (50th Percentile)', '75%': 'Q3 (75th Percentile)'})
    
    return description

train_data_numerical_A = A.original_train_data.iloc[:, 1:-1]
train_data_numerical_B = B.original_train_data.iloc[:, 1:-1]
train_data_numerical_C = C.original_train_data.iloc[:, 1:-1]

target_data_numerical_A = A.original_target_data.iloc[:, 1:]
target_data_numerical_B = B.original_target_data.iloc[:, 1:]
target_data_numerical_C = C.original_target_data.iloc[:, 1:]

### 2.1.1 Location A

In [ ]:
train_data_description_A = describe_numerical_data(train_data_numerical_A)
train_data_description_A

In [ ]:
target_data_description_A = describe_numerical_data(target_data_numerical_A)
target_data_description_A

### 2.1.2 Location B

In [ ]:
data_description_B = describe_numerical_data(train_data_numerical_B)
data_description_B

In [ ]:
target_data_description_B = describe_numerical_data(target_data_numerical_B)
target_data_description_B

### 2.1.3 Location C

In [ ]:
data_description_C = describe_numerical_data(train_data_numerical_C)
data_description_C

In [ ]:
target_data_description_C = describe_numerical_data(target_data_numerical_C)
target_data_description_C

## 2.2 Summary of general statistics
Based on general statistics, we can see that the weather data consists of both scalar and categorical values. The magnitude of the various weather data elements is relatively similar across locations. Generally, the weather data appears to be very similar across all locations. However, the PV measurements are quite different. The magnitude at location A is 5 times greater than at B and C, and there is also some difference between B and C. The number of unique measurements varies significantly, with A having many unique measurements, B fewer, and C proportionally few. It is also observed that snow density, cloud base, and ceiling height have NaN values in all datasets.

## 2.3 Individual feature analysis

### 2.3.1 PV Measurement (Target Value)  

In [ ]:
def plot_target_value(data, features, location = None):
    for feature in features:
        plt.figure(figsize=(10, 5))
        plt.plot(data['time'], data[feature], label=feature)
        plt.title(f"Time Series of {feature} - Location {location}")
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.legend()
        plt.show()
        
def plot_individual_time_series(data, features, location = None):
    for feature in features:
        plt.figure(figsize=(10, 5))
        plt.plot(data['date_forecast'], data[feature], label=feature)
        plt.title(f"Time Series of {feature} - Location {location}")
        plt.xlabel('Date')
        plt.ylabel('Value')
        plt.legend()
        plt.show()

def analyze_target_feature_with_log(target_series, feature_name):
    target_series_no_na = target_series.dropna()

    # Adding a small constant to avoid log(0)
    target_series_log_no_na = np.log(target_series_no_na + 1)

    # Basic Statistics
    stats = target_series.describe()
    stats['#NaN'] = target_series.isna().sum()
    stats['#Unique'] = target_series.nunique()

    # Visualizations
    plt.figure(figsize=(15, 5))

    # Histogram of original data
    plt.subplot(1, 3, 1)
    sns.histplot(target_series_no_na, bins=30, kde=True)
    plt.title(f'Distribution of {feature_name} Values')

    # Boxplot of original data
    plt.subplot(1, 3, 2)
    sns.boxplot(x=target_series_no_na)
    plt.title(f'Boxplot of {feature_name} Values')

    # Histogram of log-transformed data with log-normal fit
    plt.subplot(1, 3, 3)
    sns.histplot(target_series_log_no_na, bins=30, kde=True, color='skyblue')
    # ... remaining code ...
    plt.title(f'Log-Transformed Distribution of {feature_name}\n with Log-Normal Fit')

    plt.tight_layout()
    plt.show()

    return stats

def decompose_time_series_short(time_series, period, title):
    if not isinstance(time_series, pd.Series):
        time_series = pd.Series(time_series)

    # Remove any missing values from the time series
    time_series = time_series.dropna()

    # Perform STL decomposition
    stl = STL(time_series, period=period, seasonal=13, seasonal_deg=0, trend_deg=0, robust=True)
    result = stl.fit()

    # Extract the components
    trend = result.trend
    seasonal = result.seasonal
    residual = result.resid

    # Plotting the components
    plt.figure(figsize=(10, 6))

    plt.subplot(411)
    plt.plot(time_series, label='Original', color='blue')
    plt.legend(loc='upper left')
    plt.title(f'Time Series Decomposition {title}')

    plt.subplot(412)
    plt.plot(trend, label='Trend', color='red')
    plt.legend(loc='upper left')

    plt.subplot(413)
    plt.plot(seasonal, label='Seasonal', color='green')
    plt.legend(loc='upper left')

    plt.subplot(414)
    plt.plot(residual, label='Residual', color='purple')
    plt.legend(loc='upper left')

    plt.tight_layout()
    plt.show()

    components = {
        'original': time_series,
        'trend': trend,
        'seasonal': seasonal,
        'residual': residual
    }
    
    return components

def plot_feature_distributions(data, features, nrows=3, ncols=2, figsize=(14, 12), color='skyblue', log_scale=False):
    sns.set_style("whitegrid")

    # Initialize the subplot figure
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    axes = axes.flatten()

    # Loop through the features and plot their distributions
    for i, feature in enumerate(features):
        # Transform the data if log_scale is True
        axes[i].set_title(f'Distribution of Feature: {feature}', fontsize=12)

        if log_scale:
            data_to_plot = np.log1p(data[feature])
            title_suffix = ' (Log Scale)'
        else:
            data_to_plot = data[feature]
            title_suffix = ''
            
        sns.histplot(data_to_plot, kde=True, ax=axes[i], color=color, edgecolor='black')
        axes[i].set_title(f'Distribution of {feature}{title_suffix}', fontsize=12)
        axes[i].set_xlabel('')
        axes[i].set_ylabel('')

    # Adjust layout for better readability
    plt.tight_layout()
    plt.show()

def calculate_distribution_properties(data, features):
    for feature in features:
        feature_data = data[feature].dropna()
        skewness = skew(feature_data)
        kurt = kurtosis(feature_data)
        print(f"{feature}: Skewness = {skewness:.2f}, Kurtosis = {kurt:.2f}")

def plot_outliers_boxplot(data, features, location):
    num_features = len(features)
    fig, axes = plt.subplots(num_features, 1, figsize=(10, 5 * num_features), squeeze=False)
    axes = axes.flatten()

    for i, feature in enumerate(features):
        sns.boxplot(x=data[feature], ax=axes[i])
        axes[i].set_title(f'Boxplot of {feature} - Location {location}')
        axes[i].set_xlabel('Value')

    plt.tight_layout()
    plt.show()

def count_outliers_iqr(data, features):
     outlier_counts = {}
     for feature in features:
         Q1 = np.percentile(data[feature], 25)
         Q3 = np.percentile(data[feature], 75)
         IQR = Q3 - Q1
         outlier_step = 1.5 * IQR
         outliers = data[(data[feature] < Q1 - outlier_step) | (data[feature] > Q3 + outlier_step)]
         outlier_counts[feature] = len(outliers)
         
         # Print the results in a nicely formatted string
         print(f"Feature '{feature}' has {len(outliers)} outliers")
     
     return outlier_counts
 
def descriptive_statistics(data, features):
    return data[features].describe()

def plot_time_series(data, features, title):
    plt.figure(figsize=(10, 5))
    for feature in features:
        plt.plot(data['date_forecast'], data[feature], label=feature)
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

def correlation_analysis(data, features):
    correlation_matrix = data[features].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
    plt.title('Correlation Analysis')
    plt.show()
    return correlation_matrix

def missing_values_analysis(data, features):
    missing_values = data[features].isnull().sum()
    
    # Print the results in a nicely formatted string
    print("Missing Values Analysis:")
    print("-" * 30)  # Just a separator for better readability
    for feature, missing_count in missing_values.items():
        print(f"{feature}: {missing_count} missing values")
    
    return missing_values

def apply_decomposition(dataframe, feature_group, period, decompose_function=decompose_time_series_short):
    decomposed_components = {}
    
    for feature in feature_group:
        if feature in dataframe.columns:
            time_series = dataframe[feature]
            title = f'Decomposition for {feature}'
            components = decompose_function(time_series, period, title = title)
            decomposed_components[feature] = components
        else:
            print(f"Feature {feature} not found in DataFrame")
            
    return decomposed_components

def plot_autocorrelation(data, feature):
    plt.figure(figsize=(10, 5))
    autocorrelation_plot(data[feature].dropna())
    plt.title(f'Autocorrelation Plot for {feature}')
    plt.show()

def plot_partial_autocorrelation(data, feature):
    fig, ax = plt.subplots(figsize=(10, 5))
    plot_pacf(data[feature].dropna(), ax=ax)
    plt.title(f'Partial Autocorrelation Plot for {feature}')
    plt.show()

def plot_density(data, feature):
    plt.figure(figsize=(10, 5))
    sns.kdeplot(data[feature].dropna(), fill=True)
    plt.title(f'Density Plot for {feature}')
    plt.show()

def plot_lag(data, feature):
    plt.figure(figsize=(10, 5))
    lag_plot(data[feature])
    plt.title(f'Lag Plot for {feature}')
    plt.show()

def plot_rolling_statistics(data, feature, window=12):
    rolling_mean = data[feature].rolling(window=window).mean()
    rolling_std = data[feature].rolling(window=window).std()
    
    plt.figure(figsize=(14, 7))
    plt.plot(data[feature], label='Original')
    plt.plot(rolling_mean, label='Rolling Mean')
    plt.plot(rolling_std, label='Rolling Std Dev')
    plt.title(f'Rolling Mean & Standard Deviation for {feature}')
    plt.legend()
    plt.show()

def plot_qq(data, feature):
    plt.figure(figsize=(10, 5))
    probplot(data[feature].dropna(), dist="norm", plot=plt)
    plt.title(f'Q-Q Plot for {feature}')
    plt.show()

def plot_fourier_transform(data, feature):
    # Fourier Transform for finding dominant cycles
    ft = np.fft.fft(data[feature].dropna())
    frequencies = np.fft.fftfreq(len(ft))
    
    plt.figure(figsize=(14, 7))
    plt.stem(frequencies, np.abs(ft), 'b', markerfmt=" ", basefmt="-b")
    plt.title('Fourier Transform - Magnitude')
    plt.show()

def plot_ecdf(data, feature):
    sorted_var = np.sort(data[feature].dropna())
    yvals = np.arange(len(sorted_var))/float(len(sorted_var)-1)
    plt.figure(figsize=(10, 6))
    plt.plot(sorted_var, yvals)
    plt.title(f'ECDF for {feature}')
    plt.xlabel(feature)
    plt.ylabel('Cumulative Probability')
    plt.show()

def plot_periodogram(data, feature):
    f, Pxx = periodogram(data[feature].dropna())
    plt.figure(figsize=(14, 7))
    plt.semilogy(f, Pxx)
    plt.ylim([1e-4, Pxx.max() + 0.05])
    plt.title('Periodogram')
    plt.xlabel('Frequency')
    plt.ylabel('Power Spectrum')
    plt.show()

def plot_dendrogram(data, features):
    # Generate the linkage matrix
    Z = linkage(data[features].dropna(), 'ward')
    
    # Plot the dendrogram
    plt.figure(figsize=(14, 7))
    dendrogram(Z, truncate_mode='lastp', p=12, show_leaf_counts=False, leaf_rotation=90., leaf_font_size=12., show_contracted=True)
    plt.title('Hierarchical Clustering Dendrogram')
    plt.xlabel('Cluster Size')
    plt.ylabel('Distance')
    plt.show()
    
# Auxiliary function to plot partial autocorrelation
def plot_partial_autocorrelation(series):
    plt.figure(figsize=(10, 5))
    plot_pacf(series, lags=20)
    plt.title('Partial Autocorrelation Plot')
    plt.show()

def detect_anomalies_z_score(data, features, threshold=3):
    for feature in features:
        # Calculate the Z-score
        data[f'{feature}_z_score'] = np.abs(zscore(data[feature]))

        # Identify anomalies
        data[f'{feature}_anomaly'] = data[f'{feature}_z_score'] > threshold

        # Print the number of anomalies detected
        anomalies = data[f'{feature}_anomaly'].sum()
        print(f"Feature '{feature}' has {anomalies} anomalies")

    return data
    
def perform_comprehensive_analysis(df, features):
    print(descriptive_statistics(df, features))
    count_outliers_iqr(df, features)
    detect_anomalies_z_score(df, features)
    missing_values_analysis(df, features)
    try:
        plot_target_value(df, features)
    except:
        try: 
            plot_time_series(df, features)
        except Exception as e:
            print(e)
    apply_decomposition(df, features, period=24)
    calculate_distribution_properties(df, features)
    #plot_dendrogram(df, features)
    for feature in features:
        analyze_target_feature_with_log(df[feature], feature)
    #for feature in features:
    #    plot_autocorrelation(df, feature)
    #for feature in features:
    #    plot_partial_autocorrelation(df[feature])
    for feature in features:
        plot_lag(df, feature)
    for feature in features:
        plot_rolling_statistics(df, feature, window=12)
    #for feature in features:
    #    plot_qq(df, feature)
    #for feature in features:
    #    plot_fourier_transform(df, feature)
    #for feature in features:
    #    plot_ecdf(df, feature)
    #for feature in features:
    #    plot_periodogram(df, feature)

### 2.3.1.1 Location A

In [ ]:
perform_comprehensive_analysis(A.original_target_data, ['pv_measurement'])

### Location B 

In [ ]:
perform_comprehensive_analysis(B.original_target_data, ['pv_measurement'])

### Location C

In [ ]:
perform_comprehensive_analysis(C.original_target_data, ['pv_measurement'])

## Summary of PV Measurement
The first thing we notice when analyzing PV measurements is that parts of the data are characterized by constant static values. This is the case at locations B and C. Not surprisingly, there are clear trends and seasons that follow the strength of the sun throughout the year. We observe that there are mostly recorded zero values, which is expected as there is no sunlight, and thus no energy production, at night. The standard deviation and the average value follow the same trend as the measurements. There is a noticeable lag trend showing that energy production is influenced by previous energy production, which is a natural consequence of gradual weather changes and also highly dependent on previous weather conditions.

## 2.3.2 Train Features

### 2.3.2.1 Grouping Features By Correlation

In [ ]:
from typing import List

def analyze_highly_correlated_features(X: pd.DataFrame, threshold: float = 0.9) -> List[List[str]]:
    if not isinstance(X, pd.DataFrame):
        raise TypeError("X must be a pandas dataframe")

    if not isinstance(threshold, (int, float)):
        raise TypeError("threshold must be a numeric value")

    corr_matrix = X.corr().abs()

    # Get the upper triangle of the correlation matrix to avoid duplicate pairs
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Keep track of features that have already been grouped
    grouped_features = set()

    # List to store groups of correlated features
    correlated_feature_groups = []

    # Iterate through the columns of the upper triangle
    for column in upper_triangle.columns:
        # Find features that are highly correlated with the current column
        correlated_features = upper_triangle.index[upper_triangle[column] > threshold].tolist()
        if correlated_features and column not in grouped_features:
            # Add the current column to the group
            correlated_features.append(column)
            # Sort and add the group to the list of groups
            correlated_feature_group = sorted(correlated_features)
            correlated_feature_groups.append(correlated_feature_group)
            # Update the set of grouped features
            grouped_features.update(correlated_feature_group)

    # Deduplicate the list of groups
    unique_correlated_feature_groups = []
    for group in correlated_feature_groups:
        if group not in unique_correlated_feature_groups:
            unique_correlated_feature_groups.append(group)

    return unique_correlated_feature_groups

### 2.3.2.2 Location A

In [ ]:
correlation_matrix_A = train_data_numerical_A.corr()
print(correlation_matrix_A)
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix_A, annot=False, fmt=".1f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix for Dataset A')
plt.show()
correlated_feature_groups_A = analyze_highly_correlated_features(train_data_numerical_A)
correlated_feature_groups_A

### 2.3.2.3 Location B

In [ ]:
correlation_matrix_B = train_data_numerical_B.corr()
print(correlation_matrix_B)
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix_B, annot=False, fmt=".1f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix for Dataset A')
plt.show()
correlated_feature_groups_B = analyze_highly_correlated_features(train_data_numerical_B)
correlated_feature_groups_B

### 2.3.2.4 Location C

In [ ]:
correlation_matrix_C = train_data_numerical_C.corr()
print(correlation_matrix_C)
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix_C, annot=False, fmt=".1f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix for Dataset A')
plt.show()
correlated_feature_groups_C = analyze_highly_correlated_features(train_data_numerical_C)
correlated_feature_groups_C

### 2.3.3 Comparing correlated data 

In [ ]:
def compare_correlated_feature_groups(groups_A, groups_B, groups_C):
    # Converting lists of lists to lists of sets for easier comparison
    set_groups_A = [set(group) for group in groups_A]
    set_groups_B = [set(group) for group in groups_B]
    set_groups_C = [set(group) for group in groups_C]
    
    # Function to find unique and common correlated feature groups between two sets of groups
    def find_differences_and_commons(set_groups_1, set_groups_2):
        unique_to_1 = [group for group in set_groups_1 if group not in set_groups_2]
        unique_to_2 = [group for group in set_groups_2 if group not in set_groups_1]
        common_groups = [group for group in set_groups_1 if group in set_groups_2]
        return unique_to_1, unique_to_2, common_groups
    
    # Finding unique and common correlated feature groups between datasets A, B, and C
    unique_A_not_B, unique_B_not_A, common_A_B = find_differences_and_commons(set_groups_A, set_groups_B)
    unique_A_not_C, unique_C_not_A, common_A_C = find_differences_and_commons(set_groups_A, set_groups_C)
    unique_B_not_C, unique_C_not_B, common_B_C = find_differences_and_commons(set_groups_B, set_groups_C)
    
    # Calculating similarity scores
    similarity_score_A_B = len(common_A_B) / (len(set_groups_A) + len(set_groups_B) - len(common_A_B))
    similarity_score_A_C = len(common_A_C) / (len(set_groups_A) + len(set_groups_C) - len(common_A_C))
    similarity_score_B_C = len(common_B_C) / (len(set_groups_B) + len(set_groups_C) - len(common_B_C))
    
    # Preparing the results
    results = {
        "A vs B": {
            "Unique to A": unique_A_not_B,
            "Unique to B": unique_B_not_A,
            "Common": common_A_B,
            "Similarity Score": similarity_score_A_B
        },
        "A vs C": {
            "Unique to A": unique_A_not_C,
            "Unique to C": unique_C_not_A,
            "Common": common_A_C,
            "Similarity Score": similarity_score_A_C
        },
        "B vs C": {
            "Unique to B": unique_B_not_C,
            "Unique to C": unique_C_not_B,
            "Common": common_B_C,
            "Similarity Score": similarity_score_B_C
        }
    }
    
    return results

correlated_feature_comparison = compare_correlated_feature_groups(correlated_feature_groups_A, correlated_feature_groups_B, correlated_feature_groups_C)

def print_correlated_feature_comparison(results):
    for comparison, data in results.items():
        dataset_1, dataset_2 = comparison.split(" vs ")
        
        print(f"### {comparison}:\n")
        
        print(f"Unique to {dataset_1}:")
        for group in data[f"Unique to {dataset_1}"]:
            print(f"  - {sorted(list(group))}")
        print()
        
        print(f"Unique to {dataset_2}:")
        for group in data[f"Unique to {dataset_2}"]:
            print(f"  - {sorted(list(group))}")
        print()
        
        print("Common groups:")
        for group in data["Common"]:
            print(f"  - {sorted(list(group))}")
        print()
        
        print(f"Similarity Score: {data['Similarity Score']:.2f}")
        print("\n" + "-"*50 + "\n")

In [ ]:
print_correlated_feature_comparison(correlated_feature_comparison)

### 2.3.4 Groups defined by correlation and intuition

In [ ]:
radiation_and_energy = [
    'clear_sky_energy_1h:J',
    'clear_sky_rad:W',
    'diffuse_rad:W',
    'diffuse_rad_1h:J',
    'direct_rad:W',
    'direct_rad_1h:J'
]

atmospheric_conditions = [
    'absolute_humidity_2m:gm3',
    'air_density_2m:kgm3',
    'dew_point_2m:K',
    'msl_pressure:hPa',
    'pressure_100m:hPa',
    'pressure_50m:hPa',
    'sfc_pressure:hPa',
    'relative_humidity_1000hPa:p',
    't_1000hPa:K',
]

time_and_date = [
    'is_day:idx',
    'is_in_shadow:idx',
    'sun_azimuth:d',
    'sun_elevation:d'
]

cloud_cover = [
    'effective_cloud_cover:p',
    'cloud_base_agl:m',
    'total_cloud_cover:p',
    'visibility:m',
    'ceiling_height_agl:m',

]

wind_and_air_movement = [
    'wind_speed_10m:ms',
    'wind_speed_u_10m:ms',
    'wind_speed_v_10m:ms',
    'wind_speed_w_1000hPa:ms'
]

snow_features = [
    'fresh_snow_12h:cm',
    'fresh_snow_1h:cm',
    'fresh_snow_24h:cm',
    'fresh_snow_3h:cm',
    'fresh_snow_6h:cm',
    'snow_density:kgm3',
    'snow_depth:cm',
    'snow_drift:idx',
    'snow_melt_10min:mm',
    'snow_water:kgm2'
]

miscellaneous = [
    'precip_5min:mm',
    'precip_type_5min:idx',
    'prob_rime:p',
    'rain_water:kgm2',
    'super_cooled_liquid_water:kgm2',
    'dew_or_rime:idx',
    'elevation:m'
]

feature_groups = {
    'radiation_and_energy': radiation_and_energy,
    'atmospheric_conditions': atmospheric_conditions,
    'time_and_date': time_and_date,
    'cloud_cover': cloud_cover,
    'wind_and_air_movement': wind_and_air_movement,
    'snow_features': snow_features,
    'miscellaneous': miscellaneous
}

### Location A

In [ ]:
print("\nLocation A - Correlation Analysis:")
correlation_analysis(A.original_train_data, radiation_and_energy)

### Location B

In [ ]:
print("\nLocation B - Correlation Analysis:")
correlation_analysis(B.original_train_data, radiation_and_energy)

### Location C

In [ ]:
print("\nLocation C - Correlation Analysis:")
correlation_analysis(C.original_train_data, radiation_and_energy)

## Correlation Analysis Summary
We observe that the weather data are highly correlated with each other, and based on the weather phenomena they represent, clear groupings of the various data emerge. The weather phenomena follow the same correlations across the datasets, and mostly the same groupings of correlated data are formed.

## 2.5 Perform analysis on every feature

In [ ]:
def perform_analysis_by_feature_groups(feature_groups, datasets):
    for feature_name, features in feature_groups.items():
        print(f"\nAnalyzing feature group: {feature_name}")
        for dataset_name, data in datasets.items():
            print(f"\nAnalyzing dataset: {dataset_name}")
            perform_comprehensive_analysis(data, features)


datasets = {
    'Train_data_A': A.original_train_data,
    'Train_data_B': B.original_train_data,
    'Train_data_C': C.original_train_data
}

perform_analysis_by_feature_groups(feature_groups, datasets)


## 2.6 Feature Variance

In [ ]:
#Functions 
def prepare_dataframe(X: pd.DataFrame) -> pd.DataFrame:
    X_copy = X.copy()
    if TIME in X_copy.columns:
        X_copy = X_copy.drop(columns=[TIME])
    return X_copy

def analyze_low_variance_features(X: pd.DataFrame, threshold: float = 0.001) -> list:
    X = prepare_dataframe(X)
    if not isinstance(X, pd.DataFrame):
        raise TypeError("X must be a pandas dataframe")

    if not isinstance(threshold, (int, float)):
        raise TypeError("threshold must be a numeric value")
    X = X.copy()
    variances = X.var()
    low_variance_features = variances[variances < threshold].index.tolist()

    return low_variance_features

In [ ]:
feature_variance_A = analyze_low_variance_features(X = train_data_numerical_A)
feature_variance_A

In [ ]:
feature_variance_B = analyze_low_variance_features(X = train_data_numerical_B)
feature_variance_B

In [ ]:
feature_variance_C = analyze_low_variance_features(X = train_data_numerical_C)
feature_variance_C

We see that it is the same features that have low variance across the datasets. This is not unexpected, as the trend indicates that the weather data are very consistent across locations A, B, and C.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def analyze_pv_measurement_to_time(data):
    """
    Analyzes the average pv_measurement values, including hourly and monthly averages.
    Provides visualizations for patterns. Handles time data in either a column or index.
    """
    # Normalize -0 to 0
    data['pv_measurement'] = data['pv_measurement'].replace(-0.0, 0.0)
    
    # Check if 'time' column exists, else use index
    if 'time' in data.columns:
        data['hour'] = data['time'].dt.hour
        data['month'] = data['time'].dt.month
    else:  # Use index if 'time' column is not present
        data['hour'] = data.index.hour
        data['month'] = data.index.month

    # Calculate averages and plot
    hourly_avg = data.groupby('hour')['pv_measurement'].mean()
    monthly_hourly_avg = data.groupby(['month', 'hour'])['pv_measurement'].mean().unstack()

    plt.figure(figsize=(15, 7))
    sns.heatmap(monthly_hourly_avg, cmap="YlGnBu", linewidths=.5, annot=True, fmt=".0f")
    plt.title('Monthly Average PV Measurement by Hour')
    plt.xlabel('Hour of the Day')
    plt.ylabel('Month')
    plt.show()

    return hourly_avg, monthly_hourly_avg

def analyze_zero_percentage_pv_measurement_to_time(data):
    """
    Analyzes the percentage of zero pv_measurement values by hour and month.
    Handles time data in either a column or index.
    """
    # Normalize -0 to 0
    data['pv_measurement'] = data['pv_measurement'].replace(-0.0, 0.0)

    # Check if 'time' column exists, else use index
    if 'time' in data.columns:
        data['hour'] = data['time'].dt.hour
        data['month'] = data['time'].dt.month
    else:  # Use index if 'time' column is not present
        data['hour'] = data.index.hour
        data['month'] = data.index.month

    # Calculate percentages and plot
    zero_count = data[data['pv_measurement'] == 0].groupby(['month', 'hour']).size().unstack(fill_value=0)
    total_count = data.groupby(['month', 'hour']).size().unstack(fill_value=0)
    zero_percentage = (zero_count / total_count) * 100

    plt.figure(figsize=(15, 7))
    sns.heatmap(zero_percentage, cmap="viridis", linewidths=.5, annot=True, fmt=".1f")
    plt.title('Percentage of Zero PV Measurement by Hour and Month')
    plt.xlabel('Hour of the Day')
    plt.ylabel('Month')
    plt.show()

    return zero_percentage


## 2.7 Hourly PV per hour each month

### Location A

In [ ]:
hourly_avg_analysis, monthly_hourly_avg_analysis = analyze_pv_measurement_to_time(A.original_target_data)


### Location B

In [ ]:
hourly_avg_analysis, monthly_hourly_avg_analysis = analyze_pv_measurement_to_time(B.original_target_data)

### Location C

In [ ]:
hourly_avg_analysis, monthly_hourly_avg_analysis = analyze_pv_measurement_to_time(C.original_target_data)

In [ ]:
zero_percentage = analyze_zero_percentage_pv_measurement_to_time(A.original_target_data)

In [ ]:
zero_percentage = analyze_zero_percentage_pv_measurement_to_time(B.original_target_data)

In [ ]:
zero_percentage = analyze_zero_percentage_pv_measurement_to_time(C.original_target_data)


We observe that the average PV measurements follow a natural trend where they are low at night, and the periods of low measurements align with the seasons. They are strongest in the middle of the day during summer and weakest in winter and at night. The percentage of measurements that are zero follows the same trend, but we clearly see that B and C differ from A and what is expected. This is most likely a result of the presence of constant static values. It is also noted that in June, there is a value of 0.8% zero readings throughout the day at location A, which is unexpected but may stem from the same cause.

### Afther preprocessing

In [ ]:
hourly_avg_analysis, monthly_hourly_avg_analysis = analyze_pv_measurement_to_time(A.target_data)
zero_percentage = analyze_zero_percentage_pv_measurement_to_time(A.target_data)

In [ ]:
hourly_avg_analysis, monthly_hourly_avg_analysis = analyze_pv_measurement_to_time(B.target_data)
zero_percentage = analyze_zero_percentage_pv_measurement_to_time(B.target_data)

In [ ]:
hourly_avg_analysis, monthly_hourly_avg_analysis = analyze_pv_measurement_to_time(C.target_data)
zero_percentage = analyze_zero_percentage_pv_measurement_to_time(C.target_data)

There is a significant improvement in the hourly values after preprocessing. There are much fewer unexpected values, and even though the data does not exhibit exactly the desired behavior, there is a significant improvement compared to how one would expect this type of data to appear.

In [ ]:
perform_comprehensive_analysis(A.target_data, ['pv_measurement'])

In [ ]:
perform_comprehensive_analysis(B.target_data, ['pv_measurement'])

In [ ]:
perform_comprehensive_analysis(C.target_data, ['pv_measurement'])

The graphs look odd, but this is a result of having removed rows with constant values, which leads to a time jump that we plot over. However, the statistics over the data are much more as expected and smoother after preprocessing.

## 2.6 Feature Importance

In [ ]:
def feature_importance_random_forest(
            X: pd.DataFrame,
            y: pd.DataFrame,
            target_column: PV_MEASUREMENT,
            n_estimators: int = 100,
            random_state: int = 42
    ) -> list:
        X = prepare_dataframe(X)
        y = y.copy()

        model = RandomForestRegressor(
            n_estimators=n_estimators, random_state=random_state)
        model.fit(X, y)

        importances = model.feature_importances_

        importance_series = pd.Series(
            importances, index=X.columns).sort_values(ascending=False)

        return importance_series


def compute_correlation_and_matching(data1: pd.DataFrame, feature1: str,
                                         data2: pd.DataFrame, feature2: str,
                                         date_column: str) -> tuple:

        # Drop rows from data1 and data2 that don't have matching dates in both datasets
        common_dates = set(data1[date_column]).intersection(
            set(data2[date_column]))
        data1_filtered = data1[data1[date_column].isin(common_dates)]
        data2_filtered = data2[data2[date_column].isin(common_dates)]

        # Compute the correlation
        correlation = data1_filtered[feature1].corr(data2_filtered[feature2])

        # Compute the mean squared error
        mse = mean_squared_error(
            data1_filtered[feature1], data2_filtered[feature2])

        # Compute max possible error (using range of data1_filtered values as reference)
        max_possible_error = np.var(data1_filtered[feature1])

        # Compute the matching scale
        matching_scale = 1 - (mse / max_possible_error)

        return correlation, matching_scale

In [ ]:
important_features_A = feature_importance_random_forest(A.train_data, A.target_data, PV_MEASUREMENT)
important_features_A

In [ ]:
important_features_B = feature_importance_random_forest(B.train_data, B.target_data, PV_MEASUREMENT)
important_features_B

In [ ]:
important_features_C = feature_importance_random_forest(C.train_data, C.target_data, PV_MEASUREMENT)
important_features_C

By training a random forest model on the data, we can gain an understanding of which features provide the most 'knowledge' value from the data in relation to the target variable PV measurement. It appears that it is mostly the same features, or features of the same category, that have a significant impact on PV measurements. Even though this may not necessarily be the model we will use as a predictor, it serves as a good starting point for identifying which features are important. 

# 3. Model Selection

### Gradient Boosting

# Gradient Boosting Model Overview

## Basics of Gradient Boosting Model
Gradient Boosting is used for both regression and classification tasks. It builds the model stage by stage, with each stage adding a decision tree that corrects the errors made by the previous trees. 

## Key Points about Gradient Boosting
- **Type:** Supervised learning technique.
- **Suitable for:** Regression and Classification.
- **Core Concept:** Builds models sequentially, each correcting its predecessor.
- **Strength:** Reduces both bias and variance, leading to accurate predictions.
- **Overfitting:** Might be prune to overfitting.
- **Scalability:** Handles large datasets effectively.

## Implementation in Code
The `GradientBoostingModel` class encapsulates the Gradient Boosting Regressor. It initializes with  hyperparameters and includes methods for training and prediction. The `GradientBoostingPipeline` extends `IPipeline`, ensuring data validation and preprocessing. It prepares data by dropping NaNs and non-intersecting columns, and handles training and prediction tasks. The pipeline emphasizes proper data alignment and clean-up to feed into the model effectively. The hyperparameter tuning part, using `RandomizedSearchCV`, aims to find the optimal model parameters, thus enhancing model performance.


In [ ]:
class IPipeline(ABC):
    def __init__(self) -> None:
        raise NotImplementedError()

    def prepare_data_for_model(
            self,
            X: pd.DataFrame,
            X_test: pd.DataFrame,
            y: pd.DataFrame,
            y_test: pd.DataFrame = None
    ) -> tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
        raise NotImplementedError()

    def _validate_data(self, X: pd.DataFrame, y: pd.Series) -> None:
        raise NotImplementedError()

    def run(self, X_train: pd.DataFrame, y_train: pd.DataFrame, X_test: pd.DataFrame) -> None:
        raise NotImplementedError()


class GradientBoostingModel:
    __model: 'GradientBoostingRegressor'
    __predictions: np.ndarray

    def __init__(self) -> None:
        self.model = GradientBoostingRegressor(
            n_estimators=GradientBoostingHyperparameters.N_ESTIMATORS.value,
            learning_rate=GradientBoostingHyperparameters.LEARNING_RATE.value,
            max_depth=GradientBoostingHyperparameters.MAX_DEPTH.value,
            random_state=GradientBoostingHyperparameters.RANDOM_STATE.value,
        )

        self.predictions = np.array([])

    @property
    def model(self) -> 'GradientBoostingRegressor':
        return self.__model

    @model.setter
    def model(self, model: 'GradientBoostingRegressor') -> None:
        self.__model = model

    @property
    def predictions(self) -> np.ndarray:
        return self.__predictions

    @predictions.setter
    def predictions(self, predictions: np.ndarray) -> None:
        predictions = np.clip(predictions, 0, None)
        self.__predictions = predictions

    def train(self, X_train: pd.DataFrame, y_train: pd.Series) -> None:
        self.model.fit(X_train, y_train)

    def predict(self, X: pd.DataFrame) -> predictions:
        predictions: np.ndarray = self.model.predict(X)
        self.predictions = predictions

    def evaluate(
        self,
        X_test: pd.DataFrame,
        y_test: pd.Series
    ) -> tuple[Union[float, np.ndarray], Union[float, np.ndarray]]:
        y_pred = self.predict(X_test)
        mse: Union[float, np.ndarray] = mean_squared_error(
            y_ture=y_test.to_numpy(),
            y_pred=y_pred.to_numpy()
        )

        r2: Union[float, np.ndarray] = r2_score(y_test, y_pred)
        return mse, r2

    def feature_importances(self) -> np.ndarray:
        return self.model.feature_importances_


class GradientBoostingPipeline(IPipeline):
    def __init__(self) -> None:
        self.model = GradientBoostingModel()

    def _validate_data(self, X: pd.DataFrame, y: pd.Series) -> None:
        assert X.shape[0] == y.shape[0], 'X and y must have the same number of rows'
        assert X.shape[1] > 0, 'X must have at least one column'
        assert isinstance(y, pd.Series), 'y must be a pandas series'
        assert X.isna().sum().sum() == 0, 'X must not have any missing values'

    def prepare_data_for_model(
            self,
            X: pd.DataFrame,
            X_test: pd.DataFrame,
            y: pd.DataFrame,
            y_test: pd.DataFrame = None
    ) -> tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
        # X, y = self.drop_rows_with_nan_in_target_data(X=X, y=y)

        if y_test is not None:
            y_test: pd.Series = self._drop_target_data_time_column(y=y_test)

        if 'date_forecast' in X.columns:
            X = X.drop(columns=[TrainFeature.DATE.value])

        if 'date_forecast' in X_test.columns:
            X_test = X_test.drop(columns=[TrainFeature.DATE.value])

        if isinstance(y, pd.DataFrame) and TargetFeature.DATE.value in y.columns:
            y = y.drop(columns=[TargetFeature.DATE.value])

        X, X_test = PipelineUtils.remove_non_intersecting_columns(
            df1=X, df2=X_test
        )
        y: pd.Series = self._drop_target_data_time_column(y=y)
        return X, X_test, y, y_test

    def drop_rows_with_nan_in_target_data(self, X: pd.DataFrame, y: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
        y = y.dropna()
        X = X.loc[y.index]
        return X, y

    def _drop_target_data_time_column(self, y: pd.DataFrame) -> pd.Series:
        if isinstance(y, pd.DataFrame) and TargetFeature.DATE.value in y.columns:
            y.drop(columns=TargetFeature.DATE.value, axis=1, inplace=True)

        return y.squeeze()

    def run(self, train_data: pd.DataFrame, train_target_data: pd.DataFrame, test_data: pd.DataFrame) -> None:
        print("Running Gradient Boosting Pipeline")
        train_data: pd.DataFrame = train_data.copy()
        train_target_data: pd.DataFrame = train_target_data.copy()
        test_data: pd.DataFrame = test_data.copy()

        train_data, test_data, train_target_data, test_target_data = self.prepare_data_for_model(
            X=train_data,
            X_test=test_data,
            y=train_target_data
        )

        self._validate_data(X=train_data, y=train_target_data)
        print("Training model...")
        self.model.train(X_train=train_data, y_train=train_target_data)
        print("Predicting...")
        self.model.predict(X=test_data)

    # TODO: use this to find hyperparameters
    def hyperparameter_tuning(self, X_train: pd.DataFrame, y_train: pd.Series):
        param_distributions = {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.1, 1],
            'max_depth': [3, 4, 5],
        }

        randomized_search = RandomizedSearchCV(
            self.model.model,
            param_distributions=param_distributions,
            n_iter=1,  # You can adjust this
            cv=3,
            n_jobs=-1,  # Use all CPU core
            sscoring=make_scorer(
                mean_squared_error,
                greater_is_better=False
            ),
            return_train_score=True,
            random_state=42
        )  # for reproducibility)

        randomized_search.fit(X_train, y_train)

        # Update the model with the best hyperparameters
        self.model.model = randomized_search.best_estimator_

        print(f"Best parameters: {randomized_search.best_params_}")
        print(f"Best cross-validation score: {-randomized_search.best_score_}")


In [ ]:
gradient_boosting: 'GradientBoostingPipeline' = GradientBoostingPipeline()
gradient_boosting.run(
    X_train=train_data,
    y_train=target_data,
    X_test=test_data
)

# Model Stacking

Model Stacking is an technique used for regression and classification tasks. It combines multiple predictive models to improve performance. Using a meta-model to integrate predictions from base models. The meta-model is trained on the predictions of the base models. The base models are trained on the original dataset. The meta-model and base models can be any machine learning model. 

## Key Points about Model Stacking
- **Type:** Supervised learning technique.
- **Suitable for:** Regression and Classification.
- **Core Concept:** Combines multiple models’ predictions using a meta-model.
- **Strength:** Increases prediction accuracy, leveraging strengths of various models.
- **Scalability:** Efficiently handles large datasets and complex problems.

## Implementation in Code
The code implements model stacking with different models as base learners (Decision Tree, Random Forest, LGBM, CatBoost, XGB, and Gradient Boosting regressors) and LGBMRegressor as the meta-model. Each base model is configured with specific hyperparameters, and the meta-model is trained on the predictions of these base models.


In [ ]:
KAGGLE = True
SELF_MADE_SPLIT = False
HP_TUNING = False
TUNE_HP = False

In [ ]:
base_models = [
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    LGBMRegressor(),
    CatBoostRegressor(),
    XGBRegressor(),
    GradientBoostingRegressor()
]
if TUNE_HP:
    for location in location_data:
        stack = ModelStacker(None, None)

## Defining the models


In [ ]:
SEED = 42  # Example SEED value, replace with your actual seed if different

base_models = [
    DecisionTreeRegressor(
        criterion='squared_error',
        max_depth=5,
        min_samples_split=2,  # Reduced for faster training
        min_samples_leaf=1,  # Reduced for faster training
        random_state=SEED
    ),
    RandomForestRegressor(
        n_estimators=30,  # Reduced for faster training
        max_depth=15,  # Reduced for faster training
        n_jobs=-1,
        random_state=SEED
    ),
    LGBMRegressor(
        num_leaves=10,
        max_depth=5,
        verbose=-1,
        metric='squared_error',
        n_jobs=4,
        n_estimators=100,  # Reduced for faster training
        colsample_bytree=0.95,
        subsample=0.9,
        learning_rate=0.01,  # Adjusted for faster training
        random_state=SEED
    ),
    CatBoostRegressor(
        n_estimators=500,  # Reduced for faster training
        learning_rate=0.1,  # Adjusted for faster training
        thread_count=-1,
        depth=7,
        silent=True,
        bagging_temperature=0.2,
        random_seed=SEED
    ),
    XGBRegressor(
        eta=0.05,  # Adjusted for faster training
        max_depth=10,  # Reduced for faster training
        min_child_weight=1,  # Reduced for faster training
        n_estimators=200,  # Reduced for faster training
        n_jobs=-1,
        random_state=SEED
    ),
    GradientBoostingRegressor(
        learning_rate=0.01,
        n_estimators=100,  # Reduced for faster training
        subsample=0.8,  # Reduced for faster training
        criterion='friedman_mse',
        min_samples_split=2,  # Reduced for faster training
        min_samples_leaf=1,  # Reduced for faster training
        max_depth=5,  # Reduced for faster training
        random_state=SEED
    )
]

meta_model = LGBMRegressor(
    num_leaves=31,
    max_depth=-1,
    random_state=SEED,
    verbose=-1,
    metric='mse',
    n_jobs=4,
    n_estimators=100,
    colsample_bytree=1,
    subsample=1,
    learning_rate=0.1
)

### Location A


In [ ]:
if KAGGLE:
    train_data = remove_timestamp(A.train_data)
    target_data = remove_timestamp(A.target_data)
    test_data = remove_timestamp(A.test_data)
    test_target_data = None
elif SELF_MADE_SPLIT:
    train_data = remove_timestamp(A.random_train_data)
    target_data = remove_timestamp(A.random_train_target_data)
    test_data = remove_timestamp(A.random_test_data_with_gaps)
    random_test_target_data = remove_timestamp(A.random_test_target_data_with_gaps)
else:
    train_data = remove_timestamp(A.sklearn_train_data)
    target_data = remove_timestamp(A.sklearn_target_data)
    test_data = remove_timestamp(A.sklearn_test_data)
    test_target_data = remove_timestamp(A.sklearn_test_target_data)

target_data = target_data[TargetFeature.PV_MEASUREMENT.value].to_numpy()

A_stack = ModelStacker(
    base_models=base_models,
    meta_model=meta_model
)

A_stack.fit(X=train_data, y=target_data)
prediction_A = A_stack.predict(test_data)

### Location B

In [ ]:
if KAGGLE:
    train_data = remove_timestamp(B.train_data)
    target_data = remove_timestamp(B.target_data)
    test_data = remove_timestamp(B.test_data)
    test_target_data = None
elif SELF_MADE_SPLIT:
    train_data = remove_timestamp(B.random_train_data)
    target_data = remove_timestamp(B.random_train_target_data)
    test_data = remove_timestamp(B.random_test_data_with_gaps)
    random_test_target_data = remove_timestamp(B.random_test_target_data_with_gaps)
else:
    train_data = remove_timestamp(B.sklearn_train_data)
    target_data = remove_timestamp(B.sklearn_target_data)
    test_data = remove_timestamp(B.sklearn_test_data)
    test_target_data = remove_timestamp(B.sklearn_test_target_data)

target_data = target_data[TargetFeature.PV_MEASUREMENT.value].to_numpy()

B_stack = ModelStacker(
    base_models=base_models,
    meta_model=meta_model
)

B_stack.fit(X=train_data, y=target_data)
prediction_B = B_stack.predict(test_data)

### Location C

In [ ]:
if KAGGLE:
    train_data = remove_timestamp(C.train_data)
    target_data = remove_timestamp(C.target_data)
    test_data = remove_timestamp(C.test_data)
    test_target_data = None
elif SELF_MADE_SPLIT:
    train_data = remove_timestamp(C.random_train_data)
    target_data = remove_timestamp(C.random_train_target_data)
    test_data = remove_timestamp(C.random_test_data_with_gaps)
    random_test_target_data = remove_timestamp(B.random_test_target_data_with_gaps)
else:
    train_data = remove_timestamp(C.sklearn_train_data)
    target_data = remove_timestamp(C.sklearn_target_data)
    test_data = remove_timestamp(C.sklearn_test_data)
    test_target_data = remove_timestamp(C.sklearn_test_target_data)

target_data = target_data[TargetFeature.PV_MEASUREMENT.value].to_numpy()

C_stack = ModelStacker(
    base_models=base_models,
    meta_model=meta_model
)

C_stack.fit(X=train_data, y=target_data)
prediction_C = C_stack.predict(test_data)

In [ ]:
best_prediction = pd.read_csv("notebooks/best_pred.csv")["prediction"].values
if KAGGLE:
    best_pred_A = best_prediction[:720]
    best_pred_B = best_prediction[720:1440]
    best_pred_C = best_prediction[1440:]
    title = 'KAGGLE'
    comp_label = 'best prediction'
elif SELF_MADE_SPLIT:
    best_pred_A = A.random_test_target_data_with_gaps[TargetFeature.PV_MEASUREMENT.value].to_numpy()
    best_pred_B = B.random_test_target_data_with_gaps[TargetFeature.PV_MEASUREMENT.value].to_numpy()
    best_pred_C = C.random_test_target_data_with_gaps[TargetFeature.PV_MEASUREMENT.value].to_numpy()
    title = 'SELF MADE SPLIT'
    comp_label = 'random test target data'
else:
    best_pred_A = A.sklearn_test_target_data[TargetFeature.PV_MEASUREMENT.value].to_numpy()
    best_pred_B = B.sklearn_test_target_data[TargetFeature.PV_MEASUREMENT.value].to_numpy()
    best_pred_C = C.sklearn_test_target_data[TargetFeature.PV_MEASUREMENT.value].to_numpy()
    title = 'SKLEARN SPLIT'
    comp_label = 'sklearn test target data'

MAE_A = mean_absolute_error(prediction_A, best_pred_A)
MAE_B = mean_absolute_error(prediction_B, best_pred_B)
MAE_C = mean_absolute_error(prediction_C, best_pred_C)

### Plotting

In [ ]:
line_width = 1.
fig, axs = plt.subplots(3, figsize=(30, 20))
axs[0].plot(prediction_A, 'r-', label='prediction', linewidth=line_width)
axs[0].plot(best_pred_A, 'b-', label=comp_label, linewidth=line_width)
axs[0].set_title(f"[{title}] Prediction vs best prediction - MAE: {MAE_A}")
axs[0].legend()

axs[1].plot(prediction_B, 'r-', label='prediction', linewidth=line_width)
axs[1].plot(best_pred_B, 'b-', label=comp_label, linewidth=line_width)
axs[1].set_title(f"[{title}] Prediction vs best prediction - MAE: {MAE_B}")
axs[1].legend()

axs[2].plot(prediction_C, 'r-', label='prediction', linewidth=line_width)
axs[2].plot(best_pred_C, 'b-', label=comp_label, linewidth=line_width)
axs[2].set_title(f"[{title}] Prediction vs best prediction - MAE: {MAE_C}")
axs[2].legend()

plt.show()

# FLAML AutoML 

Fast and Lightweight AutoML (FLAML) is an automated machine learning framework that require minimal computational resources. FLAML automatically determines the best model and hyperparameters for a given dataset.

## Key Points about FLAML AutoML
- **Purpose:** Automated selection and tuning of machine learning models.
- **Efficiency:** Optimized for low computational resource usage.
- **Adaptability:** Works well across various types of data and ML tasks.
- **User-Friendly:** Requires minimal input from the user to start the model selection process.
- **Performance:** Competitively high accuracy with less resource consumption.
- **Flexibility:** Compatible with popular machine learning frameworks.



In [ ]:
automl = AutoML()
automl_settings = {
    "time_budget": 12000,  
    "task": "regression",  
}

train_data_A_for_flaml = train_data_A[test_data_A.columns.intersection(train_data_A.columns)]
        
train_data_A_for_flaml = train_data_A_for_flaml.copy().drop(columns=['date_forecast'])
test_data_A_for_flaml = test_data_A.copy().drop(columns=['date_forecast'])

train_data_A_for_flaml.columns = ['feature_' + str(i) for i in range(len(train_data_A_for_flaml.columns))]
target_data_A.columns = ['target_' + str(i) for i in range(len(target_data_A.columns))]
test_data_A_for_flaml.columns = ['feature_' + str(i) for i in range(len(test_data_A_for_flaml.columns))]

test_data_A_for_flaml = test_data_A_for_flaml.drop(columns=['feature_11'])

automl.fit(X_train=train_data_A_for_flaml, y_train=target_data_A['target_1'], **automl_settings)
new_model = automl.model
joblib.dump(new_model, 'new_model_A.pkl')

#loaded_model_A = joblib.load('/Users/eirikvarnes/ML/Modeler/FLAML/best_model_A.pkl')
predictions_A: np.ndarray = new_model.predict(test_data_A_for_flaml)
clipped_predictions_A = np.clip(predictions_A, 0, None)

clipped_predictions_A

In [ ]:
combined_predictions = np.concatenate([clipped_predictions_A, clipped_predictions_B, clipped_predictions_C])
combined_predictions

# H2o AutoML 
A autoML that is more advanced than FLAML, but also more resource intensive.

In [ ]:
def H2O_train(location):
    h2o.init()
    train_data = location.train_data.copy()
    target_data = location.target_data.copy()
    
    merged_data = pd.merge(left=train_data, right=target_data, left_index=True, right_index=True)
    
    h2o_data = h2o.H2OFrame(merged_data)
    
    # Identify predictors and response
    x = h2o_data.columns
    y = "pv_measurement"
    x.remove(y)    
    
    aml_A = H2OAutoML(
        max_runtime_secs = 7200, 
        seed = 42, 
        sort_metric = "MAE"
    )
    
    aml_A.train(x = x, y = y, training_frame = h2o_data)
    
    h2o.save_model(model=aml_A.leader, path=f"/Users/eirikvarnes/ML/Modeler/H2O/endgame", force=True)

# CatBoost 

CatBoost is a library for gradient boosting on decision trees.

## Key Points about CatBoost
- **Type:** Supervised learning algorithm.
- **Suitable for:** Classification and Regression.
- **Core Concept:** Utilizes gradient boosting on decision trees, excelling with categorical data.
- **Strength:** Delivers high performance with minimal need for data preprocessing, reduces overfitting.
- **Handling Categorical Data:** Effectively processes categorical features without extensive preprocessing.
- **Speed and Scalability:** Fast training and prediction, scales well with large datasets.


### Tuning 

In [ ]:
def grid_search_hyper_parameter_tuning(location: LocationData) -> None:
    catboost_model = CatBoostRegressor(
        iterations=20000,
        loss_function='MAE',
        od_type='Iter',
        od_wait=100,
        verbose=True
    )

    param_grid = {
        'depth': [6, 8, 10],
        'learning_rate': [0.1, 0.01, 0.001]
    }

    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
    grid_search = GridSearchCV(estimator=catboost_model, param_grid=param_grid, scoring=mae_scorer, cv=3, verbose=1)
    X_train_split, X_validation_split, y_train_split, y_validation_split = train_test_split(
        location.train_data,
        location.target_data,
        test_size=0.2,
        random_state=42
    )
    grid_search.fit(X_train_split, y_train_split)

    print(f"Best parameters found: {grid_search.best_params_}")
    print(f"Best MAE score from grid search: {-grid_search.best_score_}")


def random_search_hyper_parameter_tuning(location: LocationData) -> None:
    catboost_model = CatBoostRegressor(
        iterations=20000,
        loss_function='MAE',
        od_type='Iter',
        od_wait=100,
        verbose=True
    )

    param_distributions = {
        'depth': [6, 8, 10, 12],
        'learning_rate': [0.1, 0.01, 0.001, 0.0001],
    }

    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

    random_search = RandomizedSearchCV(
        estimator=catboost_model,
        param_distributions=param_distributions,
        n_iter=1,
        scoring=mae_scorer,
        cv=3,
        random_state=42,
        verbose=1
    )

    X_train_split, X_validation_split, y_train_split, y_validation_split = train_test_split(
        location.train_data,
        location.target_data,
        test_size=0.2,
        random_state=42
    )

    random_search.fit(X_train_split, y_train_split)

    print(f"Best parameters found: {random_search.best_params_}")
    print(f"Best MAE score from random search: {-random_search.best_score_}")

In [ ]:
def evaluate_model(model, X_val, y_val):
    predictions = model.predict(X_val)
    mae = mean_absolute_error(y_val, predictions)
    return mae

# Without eval

In [ ]:
def train_catboost(location, params):
    N_MODELS = 1
    validation_size = 0.2

    params['verbose'] = False
    params['loss_function'] = 'MAE'
    params['eval_metric'] = 'MAE'
    params['random_seed'] = 42
    params['iterations'] = 20_000
    params['per_float_feature_quantization'] = ['20:border_count=1024']
    # params['has_time'] = True
    # params['bootstrap_type'] = 'Bayesian'
    # params['boosting_type'] = 'Plain'
    # params['bagging_temperature'] = 3.65

    train_data = location.train_data
    target_data = location.target_data

    # Calculate the index for splitting the data
    split_index = int((1 - validation_size) * len(train_data))

    # Split the data into training and validation sets
    X_train, y_train = train_data.iloc[:split_index, :], target_data.iloc[:split_index]
    X_val, y_val = train_data.iloc[split_index:, :], target_data.iloc[split_index:]

    models = []
    """
    depth_values = np.random.randint(6, 12, size=N_MODELS)
    learning_rate_values = 10 ** -np.random.uniform(1, 2, size=N_MODELS)
    learning_rates = params['learning_rate']
    depths = params['depth']
    """

    for i in range(N_MODELS):
        """
        params['depth'] = depth_values[i]
        params['learning_rate'] = learning_rate_values[i]
        """
        model = CatBoostRegressor(**params)

        # Fit the model using the validation set for early stopping
        model.fit(
            X_train,
            y_train,
            eval_set=(X_val, y_val),
            use_best_model=True,
            early_stopping_rounds=100
        )

        # After training, evaluate the final model on the validation set
        final_mae = evaluate_model(model, X_val, y_val)
        print(f"Final MAE for model {i + 1}: {final_mae}")

        # Store the model
        models.append(model)

    return models

# With eval

In [ ]:
def train_catboost_ensemble(location, N_MODELS=10):
    train_data = location.train_data
    target_data = location.target_data

    total_length = len(train_data)
    val_size = int(0.1 * total_length)

    mae_list = []
    models = []
    depth_values = [7, 7, 8, 8, 8, 8, 9, 9, 9, 10]
    learning_rate_values = [0.01, 0.04, 0.01, 0.02, 0.03, 0.04, 0.02, 0.03, 0.04, 0.03]
    summer_evaluated = []
    train_dates = train_data.index.values

    for i in range(N_MODELS):
        val_start_idx = int((i / N_MODELS) * (total_length - val_size))

        X_train = pd.concat([train_data[:val_start_idx], train_data[val_start_idx + val_size:]], axis=0)
        y_train = pd.concat([target_data[:val_start_idx], target_data[val_start_idx + val_size:]], axis=0)
        X_val = train_data[val_start_idx:val_start_idx + val_size]
        y_val = target_data[val_start_idx:val_start_idx + val_size]

        date_at_middle_of_val = pd.to_datetime(X_train.index[val_start_idx + int(val_size / 2)])
        evaluation_month = date_at_middle_of_val.month + 7
        if evaluation_month > 12:
            evaluation_month -= 12

        summer_months = [5, 6, 7]
        is_summer = evaluation_month in summer_months
        summer_evaluated.append(is_summer)

        model = CatBoostRegressor(
            iterations=20000,
            depth=depth_values[i],
            learning_rate=learning_rate_values[i],
            loss_function='MAE',
            od_type='Iter',
            od_wait=100,
            l2_leaf_reg=3,
            random_seed=42,
            verbose=False,
        )

        # print(f"Model nr: {i + 1}")
        # print(f"Evaluation month: {evaluation_month}")
        # print(f"Train data index: {[0, val_start_idx - 1]} and {[val_start_idx + val_size, total_length - 1]}")
        # print(f"Validation data index {[val_start_idx, val_start_idx + val_size - 1]}")

        # Fit the model using the validation set for early stopping
        model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, early_stopping_rounds=100)

        # After training, evaluate the final model on the validation set
        final_mae = evaluate_model(model, X_val, y_val)
        # print(f"Final MAE for model {i + 1}: {final_mae}")
        print(f"Model {i + 1} M[{evaluation_month}]: MAE:\t{round(final_mae, 4)}")

        mae_list.append(final_mae)

        models.append(model)

    print(f"Average MAE:\t{round(np.mean(mae_list), 4)}")

    return models, summer_evaluated


def get_ensemble_predictions(models, test_data, summer_evaluated):
    all_predictions = []

    for i, model in enumerate(models):
        predictions = model.predict(test_data)
        if summer_evaluated[i]:
            all_predictions.extend([
                predictions,
                predictions,
                predictions,
                predictions,
            ])
        else:
            all_predictions.append(predictions)

    ensemble_predictions = np.mean(all_predictions, axis=0)
    return ensemble_predictions

```python
def hei() -> None
```